<a href="https://colab.research.google.com/github/Qteit/Prediction-of-Product-Sales/blob/main/project_1_core_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# part 5

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(transform_output='pandas')

In [2]:
path = '/content/drive/MyDrive/AXSOSACADEMY/AXSOSACADEMY/01-Fundamentals/Week02/sales_predictions_2023.csv'
df=pd.read_csv(path)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


***Cleaning the Data***







In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [4]:
df.duplicated().sum()

0

In [5]:
df.isna().sum()

,0
Item_Identifier,0
Item_Weight,1463
Item_Fat_Content,0
Item_Visibility,0
Item_Type,0
Item_MRP,0
Outlet_Identifier,0
Outlet_Establishment_Year,0
Outlet_Size,2410
Outlet_Location_Type,0


In [16]:
cat_col = df.select_dtypes(include='object').columns
if 'Item_Identifier' in df.columns:
    df = df.drop(columns=['Item_Identifier'], axis=1)
    print("Column 'Item_Identifier' dropped successfully.")
else:
    print("Column 'Item_Identifier' not found in DataFrame.")
cat_col

Column 'Item_Identifier' not found in DataFrame.


Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [17]:
for col in cat_col:
  print(f'{col}: {df[col].value_counts()}')
  print('\n')

Item_Fat_Content: Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64


Item_Type: Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: count, dtype: int64


Outlet_Identifier: Outlet_Identifier
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: count, dtype: int64


Outlet_Size: Outlet_Size
Medium    2793
Small     2388
High       932
Name: count, dtype: int64


Outlet_Location_Type: Outlet_Location_Type
Tier 3    3350
Tier 2    2785


In [8]:
ord_col=['Outlet_Location_Type','Outlet_Size','Item_Fat_Content']

In [9]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF':"Low Fat",'reg':'Regular' , "low fat": "Low Fat"})
df['Item_Fat_Content'].value_counts()

,count
Item_Fat_Content,
Low Fat,5517
Regular,3006


In [10]:
y=df['Item_Outlet_Sales']
x=df.drop(columns=['Item_Outlet_Sales'])
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)


In [11]:
num_col=x_train.select_dtypes(include='number').columns
print('numaric columns:',num_col)
scaler=StandardScaler()
impute = SimpleImputer(strategy = 'median')
num_pipi = make_pipeline(impute,scaler)
num_pipi

numaric columns: Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [12]:
ord_col=['Outlet_Location_Type','Outlet_Size','Item_Fat_Content']
ord_impute = SimpleImputer(strategy='most_frequent')
qual_ordinal_location=('Tier 1','Tier 2','Tier 3')
qual_ordinal_size=('Small','Medium','High')
qual_ordinal_fat=('Low Fat','Regular')
ordinal_category_orders=[qual_ordinal_location , qual_ordinal_size , qual_ordinal_fat]
ord_scaler=StandardScaler()
ord_encoder=OrdinalEncoder(categories=ordinal_category_orders)
ord_pipi=make_pipeline(ord_impute,ord_encoder,ord_scaler)
ord_pipi

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[('Tier 1', 'Tier 2', 'Tier 3'),
                                            ('Small', 'Medium', 'High'),
                                            ('Low Fat', 'Regular')])),
                ('standardscaler', StandardScaler())])

In [18]:
ohe_col=x_train[cat_col].drop(columns=ord_col).columns
print('nominal columns :',ohe_col)
ohe_impute=SimpleImputer(strategy='most_frequent')
ohe_encoder=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
ohe_pipi=make_pipeline(ohe_impute,ohe_encoder)
ohe_pipi

nominal columns : Index(['Item_Type', 'Outlet_Identifier', 'Outlet_Type'], dtype='object')


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [14]:
num_tuple=('numaric',num_pipi, num_col)
num_tuple

('numaric',
 Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                 ('standardscaler', StandardScaler())]),
 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
        'Outlet_Establishment_Year'],
       dtype='object'))

In [19]:
ord_tuple=('ordinal',ord_pipi,ord_col)
ord_tuple

('ordinal',
 Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                 ('ordinalencoder',
                  OrdinalEncoder(categories=[('Tier 1', 'Tier 2', 'Tier 3'),
                                             ('Small', 'Medium', 'High'),
                                             ('Low Fat', 'Regular')])),
                 ('standardscaler', StandardScaler())]),
 ['Outlet_Location_Type', 'Outlet_Size', 'Item_Fat_Content'])

In [20]:
ohe_tuple=('categorical',ohe_pipi,ohe_col)
ohe_tuple

('categorical',
 Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                 ('onehotencoder',
                  OneHotEncoder(handle_unknown='ignore', sparse_output=False))]),
 Index(['Item_Type', 'Outlet_Identifier', 'Outlet_Type'], dtype='object'))

In [25]:
preprocessor = ColumnTransformer([num_tuple, ord_tuple, ohe_tuple],
                                    verbose_feature_names_out=False)
preprocessor

ColumnTransformer(transformers=[('numaric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                ('ordinal',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  Ord...
                                                  StandardScaler())]),
                                 ['Outlet_Location_Type', 'Outlet_Size',
                                  'Item_Fat_Content']),
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['Item_Type', 'Outlet_Identifier', 'Outlet_Type'], dtype='object'))],
                  verbose_feature_names_out=False)

In [26]:
preprocessor.fit(x_train)

ColumnTransformer(transformers=[('numaric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                ('ordinal',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  Ord...
                                                  StandardScaler())]),
                                 ['Outlet_Location_Type', 'Outlet_Size',
                                  'Item_Fat_Content']),
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['Item_Type', 'Outlet_Identifier', 'Outlet_Type'], dtype='object'))],
                  verbose_feature_names_out=False)

In [27]:
x_train_processed_demo=preprocessor.transform(x_train)
x_train_processed_demo.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Size,Item_Fat_Content,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,...,Outlet_Identifier_OUT019,Outlet_Identifier_OUT027,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
4776,0.827485,-0.712775,1.828109,1.327849,1.084948,0.287374,-0.740321,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7510,0.566644,-1.291052,0.603369,1.327849,1.084948,0.287374,1.350766,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5828,-0.121028,1.813319,0.244541,0.136187,-1.384777,0.287374,1.350766,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
5327,-1.158464,-1.004931,-0.952591,0.732018,-0.149914,-1.384048,-0.740321,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4810,1.538870,-0.965484,-0.336460,0.493686,-0.149914,0.287374,-0.740321,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


# part 6

## Linear Regression Model

In [28]:
# Build a linear regression model.
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
# Fit linear regression
linreg_pipe = make_pipeline(preprocessor,LinearRegression())
linreg_pipe.fit(x_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numaric',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                                 ('ordinal',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='m...
                                                   'Outlet_Size',
                                                   'Item_Fat_Content']),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['Item_Type', 'Outlet_Identifier', 'Outlet_Type'], dtype='object'))],
                                   verbose_feature_names_out=False)),
                ('linearregression', LinearRegression())])

In [30]:
# Get predictions
y_hat_train = linreg_pipe.predict(x_train)
y_hat_test = linreg_pipe.predict(x_test)

In [40]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

def evaluate_model(y_true, y_pred, split='training'):
  """ prints RMSE, and R2 metrics, include which data split was evaluated

  Args:
    y_true: y-train or y-test
    y_pred: result of model.predict(X)
    split: which data split is being evaluate ['training','test']
  """

  r2 = r2_score(y_true,y_pred)
  mae = mean_absolute_error(y_true,y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = mean_squared_error(y_true,y_pred,squared=False)


  print(f'Results for {split} data:')
  print(f"  - R^2 = {round(r2,3)}")
  print(f"  - MAE = {round(mae,3)}")
  print(f"  - MSE = {round(mse,3)}")
  print(f"  - RMSE = {round(rmse,3)}")

In [41]:
from sklearn.metrics import mean_squared_error, r2_score

def evaluate_regression(model, X_train, y_train, X_test, y_test):
    """
    Evaluates a regression model using common metrics.

    Args:
        model: The fitted regression model.
        X_train: The training data features.
        y_train: The training data target variable.
        X_test: The testing data features.
        y_test: The testing data target variable.

    Prints:
        - R-squared score on training and testing data.
        - Mean Squared Error on training and testing data.
    """
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    print("Training Set:")
    print(f"R-squared: {r2_score(y_train, y_train_pred):.4f}")
    print(f"Mean Squared Error: {mean_squared_error(y_train, y_train_pred):.4f}")

    print("\nTesting Set:")
    print(f"R-squared: {r2_score(y_test, y_test_pred):.4f}")
    print(f"Mean Squared Error: {mean_squared_error(y_test, y_test_pred):.4f}")

In [42]:
## Evaluate model's performance
evaluate_model(y_train, y_hat_train,split='training')
evaluate_model(y_test, y_hat_test,split='testing')

Results for training data:
  - R^2 = 0.562
  - MAE = 847.155
  - MSE = 1297559.084
  - RMSE = 1139.105
Results for testing data:
  - R^2 = 0.567
  - MAE = 804.101
  - MSE = 1194355.559
  - RMSE = 1092.866


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Random Forest model

In [43]:
from sklearn.ensemble import RandomForestRegressor

In [45]:
# Pipeline and fitting the random forest
rf_tree_pipe = make_pipeline(preprocessor,RandomForestRegressor(random_state = 42))
rf_tree_pipe.fit(x_train, y_train)

# Getting predictions for training and test data
y_hat_train = rf_tree_pipe.predict(x_train)
y_hat_test = rf_tree_pipe.predict(x_test)

In [46]:
# Evaluate Performance
evaluate_model(y_train, y_hat_train,split='training')
evaluate_model(y_test, y_hat_test,split='testing')

Results for training data:
  - R^2 = 0.938
  - MAE = 296.847
  - MSE = 183065.54
  - RMSE = 427.862
Results for testing data:
  - R^2 = 0.558
  - MAE = 766.875
  - MSE = 1218973.946
  - RMSE = 1104.072


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Compare the training vs. test R-squared values and answer the question: to what extent is this model overfit/underfit?


*   It's good for the training data but not very well for the testing data

Compare this model's performance to the linear regression model: which model has the best test scores?

*   Eventhough the model have bias, it's better than the linear regression modle


## GridSearch CV

In [47]:
#Use GridSearchCV to tune at least two hyperparametersUse
rf_tree_pipe.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('numaric',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='median')),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
          'Outlet_Establishment_Year'],
         dtype='object')),
                                   ('ordinal',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('ordinalencoder',
                                                     Ord...
                                                     StandardScaler())]),
                           

In [48]:
# Identifying Parametars
params = {'randomforestregressor__max_depth': [10, None],
          'randomforestregressor__n_estimators':[50, 100],
          'randomforestregressor__min_samples_leaf':[1, 2, 3],
          'randomforestregressor__max_features':[1.0, 5.0 ],
          'randomforestregressor__oob_score':[False]
          }

In [50]:
# Importing the GridSearchCV class from sklearn.model_selection
from sklearn.model_selection import GridSearchCV

# Identifting GridSearch
gridsearch = GridSearchCV (rf_tree_pipe, params, n_jobs=-1, cv=3,  verbose=1)
gridsearch.fit(x_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
36 fits failed out of a total of 72.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
36 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/usr/local/lib/pyth

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('numaric',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                                                        ('ordinal',
                                                                         Pipeline(steps=[('simpleimpute...
                                                          verbose_feature_names_out=False)),
                                       ('randomforestregressor',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'randomforestregressor__max_depth': [10, None],
                         'randomforestregressor__max_features': [1.0, 5.0],
                         'randomforestregressor__min_samples_leaf': [1, 2, 3],
                         'randomforestregressor__n_estimators': [50, 100],
                         'randomforestregressor__oob_score': [False]},
             verbose=1)

In [51]:
# Obtain best parameters
gridsearch.best_params_

{'randomforestregressor__max_depth': 10,
 'randomforestregressor__max_features': 1.0,
 'randomforestregressor__min_samples_leaf': 1,
 'randomforestregressor__n_estimators': 100,
 'randomforestregressor__oob_score': False}

In [53]:
# Define and refit best model
best_rf = gridsearch.best_estimator_
evaluate_regression(best_rf, x_train, y_train, x_test, y_test)

Training Set:
R-squared: 0.7219
Mean Squared Error: 823157.4601

Testing Set:
R-squared: 0.5911
Mean Squared Error: 1128093.5085


# CRISP-DM Phase 5 - Evaluation

## Recommended Model

- Overall, which model do you recommend?

the random forest model shows a strong training R2 of 0.936, meaning it captures a significant amount of variance in the training data.
although its testing R2 is lower at 0.546, it still significantly outperforms the Linear regression model, which struggles with a negative R2 on the testing set, this suggests the random forest model has better predictive capabilities for new data.

## Explanation for Stakeholders

- Understanding r-squared

the testing R2 of 0.546 for the random forest model indicates that approximately 54.6% of the variablity in the outcome can be accounted for by the model's features. This suggests that the model is somewhat effective, but there is potential for further improvement.

- selected metric: RMSE

reason for Choosing rmse: The root mean squared error (RMSE) is selected as it expresses the model's prediction errors in the same units as the target variable. The Random Forest model has an RMSE of 1128.204 on the testing set, implying that, on average, the model's predictions differ from the actual values by around 1128 units.

##Overfitting vs. Underfitting

- Comparison of Training and Testing Performance

 random forest training R2: 0.936

 random rorest testing R2: 0.546

the noticeable gap between the training and testing R2 values suggests that the random forest model may be overfitting the training data, meaning it performs exceptionally well on that data but not as well on new, unseen data however it still demonstrates much better overall performance compared to the Linear Regression model, which fails to generalize effectively.

##Conclusion

in conclusion, the random forest model is recommended for implementation, While it does exhibit some signs of overfitting its performance is significantly stronger than that of the linear regression model with some potential refinements and feature adjustments, the random forest model can likely be improved for better predictions on unseen data.# New Section